In [37]:
import json
from PIL import Image, ImageDraw
import numpy as np
import os
import shutil

新增資料夾

In [72]:
img_folder = 'output_dir/images'
input_folders = ['./data_20231207/Cancer', './data_20231207/Mix', './data_20231207/Warthin']
output_folders = ['output_dir/masks/0','output_dir/masks/1','output_dir/masks/2']
output_folders_new = 'output_dir/new_mask'

test_images_folders = 'output_dir_test/images'
test_input_folders = './demo_test9'
test_output_folders = ['output_dir_test/masks/0','output_dir_test/masks/1','output_dir_test/masks/2']

os.makedirs(img_folder, exist_ok=True)
os.makedirs(output_folders_new, exist_ok=True)
for folder in output_folders:
    os.makedirs(folder, exist_ok=True)

os.makedirs(test_images_folders, exist_ok=True)
for folder in test_output_folders:
    os.makedirs(folder, exist_ok=True)

複製所有照片到img資料夾

In [51]:
# 将每个 PNG 图片从 A 资料夹复制到 B 资料夹
def copy_img(input_f, img_f):
    for idx, folder in enumerate(input_f if isinstance(input_f, list) else [input_f]):
        files_in_folder = os.listdir(folder)
        # 获取该文件夹中的所有 .png 文件
        png_files = [file for file in files_in_folder if file.endswith('.png')]
        for file in png_files:
            src_path = os.path.join(folder, file)
            dst_path = os.path.join(img_f, file)
            shutil.copy(src_path, dst_path)
            print(f"已复制 {file} 到 {img_f}")

    print("复制完成")

In [56]:
copy_img(input_folders, img_folder)

已复制 c103-10027933_1.png 到 output_dir/images
已复制 c103-10027933_2.png 到 output_dir/images
已复制 c103-10027933_3.png 到 output_dir/images
已复制 c189-03816098_14.png 到 output_dir/images
已复制 c189-03816098_15.png 到 output_dir/images
已复制 c189-03816098_16.png 到 output_dir/images
已复制 c189-03816098_17.png 到 output_dir/images
已复制 c189-03816098_18.png 到 output_dir/images
已复制 c25-06905357_10.png 到 output_dir/images
已复制 c25-06905357_11.png 到 output_dir/images
已复制 c25-06905357_12.png 到 output_dir/images
已复制 c25-06905357_13.png 到 output_dir/images
已复制 c278-13836909_5.png 到 output_dir/images
已复制 c278-13836909_6.png 到 output_dir/images
已复制 c278-13836909_7.png 到 output_dir/images
已复制 c278-13836909_8.png 到 output_dir/images
已复制 c278-13836909_9.png 到 output_dir/images
已复制 c30-15762303_10.png 到 output_dir/images
已复制 c30-15762303_11.png 到 output_dir/images
已复制 c30-15762303_12.png 到 output_dir/images
已复制 c30-15762303_6.png 到 output_dir/images
已复制 c30-15762303_7.png 到 output_dir/images
已复制 c30-15762303_8.png 到 outp

Json 轉 mask(png)

In [40]:
def add_black_image(data,folder_paths,image_filename):
    for folder_path  in enumerate(folder_paths):
        black_img = Image.new('L', (data['imageWidth'], data['imageHeight']), 0)
        # black = ImageDraw.Draw(black_img)
        b_mask_np = np.array(black_img)
        if folder_path[1] == './data_20231207/Mix':
            output_f = 'output_dir/masks/1'   
        elif folder_path[1] == './data_20231207/Cancer':
            output_f = 'output_dir/masks/0'
        else:
            output_f = 'output_dir/masks/2'
        b_output_mask_path = os.path.join(output_f, image_filename)
        print(b_output_mask_path)
        Image.fromarray(b_mask_np).save(b_output_mask_path)
        print(f"黑色 mask 已保存 {b_output_mask_path}")


In [71]:
# 迴圈處理每個資料夾中的圖片和對應的 JSON 檔案
for current_index, (input_folder, output_folder) in enumerate(zip(input_folders, output_folders)):
    other_folders = input_folders[:current_index] + input_folders[current_index + 1:]
    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            json_path = os.path.join(input_folder, filename)
            image_filename = filename.replace('.json', '.png')
            image_path = os.path.join(input_folder, image_filename)
            
            # 讀取 JSON 檔案
            with open(json_path, 'r') as file:
                data = json.load(file)

            add_black_image(data,other_folders,image_filename)

            # 創建一個和圖像大小相同的全黑圖像
            image = Image.new('L', (data['imageWidth'], data['imageHeight']), 0)
            draw = ImageDraw.Draw(image)

            # 繪製每個形狀（即多邊形）
            for shape in data['shapes']:
                label = shape['label']
                if label == "left normal" or label == "right normal":
                    continue
                points = shape['points']
                polygon_points = [tuple(point) for point in points]
                draw.polygon(polygon_points, fill=255)  # 在 mask 圖像上繪製多邊形
                
            # 將 PIL 圖像轉換為 NumPy array
            mask_np = np.array(image)

            # 輸出的 mask 檔案路徑
            output_mask_path = os.path.join(output_folder, image_filename)

            # 將 NumPy array 保存為 PNG 檔案
            Image.fromarray(mask_np).save(output_mask_path)
            print(f"已保存 {output_mask_path}")


print("完成轉換")

output_dir/masks/1\c103-10027933_1.png
黑色 mask 已保存 output_dir/masks/1\c103-10027933_1.png
output_dir/masks/2\c103-10027933_1.png
黑色 mask 已保存 output_dir/masks/2\c103-10027933_1.png
已保存 output_dir/masks/0\c103-10027933_1.png
output_dir/masks/1\c103-10027933_2.png
黑色 mask 已保存 output_dir/masks/1\c103-10027933_2.png
output_dir/masks/2\c103-10027933_2.png
黑色 mask 已保存 output_dir/masks/2\c103-10027933_2.png
已保存 output_dir/masks/0\c103-10027933_2.png
output_dir/masks/1\c103-10027933_3.png
黑色 mask 已保存 output_dir/masks/1\c103-10027933_3.png
output_dir/masks/2\c103-10027933_3.png
黑色 mask 已保存 output_dir/masks/2\c103-10027933_3.png
已保存 output_dir/masks/0\c103-10027933_3.png
output_dir/masks/1\c189-03816098_14.png
黑色 mask 已保存 output_dir/masks/1\c189-03816098_14.png
output_dir/masks/2\c189-03816098_14.png
黑色 mask 已保存 output_dir/masks/2\c189-03816098_14.png
已保存 output_dir/masks/0\c189-03816098_14.png
output_dir/masks/1\c189-03816098_15.png
黑色 mask 已保存 output_dir/masks/1\c189-03816098_15.png
output_dir/

單純 mask 沒有產生黑色圖片

In [73]:
# 迴圈處理每個資料夾中的圖片和對應的 JSON 檔案
for current_index, input_folder in enumerate(input_folders):
    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            json_path = os.path.join(input_folder, filename)
            image_filename = filename.replace('.json', '.png')
            image_path = os.path.join(input_folder, image_filename)
            
            # 讀取 JSON 檔案
            with open(json_path, 'r') as file:
                data = json.load(file)

            # 創建一個和圖像大小相同的全黑圖像
            image = Image.new('L', (data['imageWidth'], data['imageHeight']), 0)
            draw = ImageDraw.Draw(image)

            # 繪製每個形狀（即多邊形）
            for shape in data['shapes']:
                label = shape['label']
                if label == "left normal" or label == "right normal":
                    continue
                points = shape['points']
                polygon_points = [tuple(point) for point in points]
                draw.polygon(polygon_points, fill=255)  # 在 mask 圖像上繪製多邊形
                
            mask_np = np.array(image)

            output_mask_path = os.path.join(output_folders_new, image_filename)

            Image.fromarray(mask_np).save(output_mask_path)
            print(f"已保存 {output_mask_path}")


print("完成轉換")

已保存 output_dir/new_mask\c103-10027933_1.png
已保存 output_dir/new_mask\c103-10027933_2.png
已保存 output_dir/new_mask\c103-10027933_3.png
已保存 output_dir/new_mask\c189-03816098_14.png
已保存 output_dir/new_mask\c189-03816098_15.png
已保存 output_dir/new_mask\c189-03816098_16.png
已保存 output_dir/new_mask\c189-03816098_17.png
已保存 output_dir/new_mask\c189-03816098_18.png
已保存 output_dir/new_mask\c25-06905357_10.png
已保存 output_dir/new_mask\c25-06905357_11.png
已保存 output_dir/new_mask\c25-06905357_12.png
已保存 output_dir/new_mask\c25-06905357_13.png
已保存 output_dir/new_mask\c278-13836909_5.png
已保存 output_dir/new_mask\c278-13836909_6.png
已保存 output_dir/new_mask\c278-13836909_7.png
已保存 output_dir/new_mask\c278-13836909_8.png
已保存 output_dir/new_mask\c278-13836909_9.png
已保存 output_dir/new_mask\c30-15762303_10.png
已保存 output_dir/new_mask\c30-15762303_11.png
已保存 output_dir/new_mask\c30-15762303_12.png
已保存 output_dir/new_mask\c30-15762303_6.png
已保存 output_dir/new_mask\c30-15762303_7.png
已保存 output_dir/new_mask\c30-1

test 那9張 產生mask

In [57]:
copy_img(test_input_folders, test_images_folders)

已复制 b58-15991078_11.png 到 output_dir_test/images
已复制 b59-15762303_9.png 到 output_dir_test/images
已复制 b60-02986412_8.png 到 output_dir_test/images
已复制 b61-08688114_6.png 到 output_dir_test/images
已复制 b70-09071962_9.png 到 output_dir_test/images
已复制 b72-04955055_3.png 到 output_dir_test/images
已复制 c230-11996869_12.png 到 output_dir_test/images
已复制 c281-17880678_9.png 到 output_dir_test/images
已复制 c283-17842193_16.png 到 output_dir_test/images
复制完成


In [70]:
# for current_index, output_folder in enumerate(test_output_folders):
for filename in os.listdir(test_input_folders):
    if filename.endswith('.json'):
        json_path = os.path.join(test_input_folders, filename)
        image_filename = filename.replace('.json', '.png')
        image_path = os.path.join(test_input_folders, image_filename)
        
        # 讀取 JSON 檔案
        with open(json_path, 'r') as file:
            data = json.load(file)

        
        # 創建一個和圖像大小相同的全黑圖像
        image = Image.new('L', (data['imageWidth'], data['imageHeight']), 0)
        draw = ImageDraw.Draw(image)

        # 繪製每個形狀（即多邊形）
        for shape in data['shapes']:
            label = shape['label']
            if label == "left normal" or label == "right normal":
                continue
            points = shape['points']
            polygon_points = [tuple(point) for point in points]
            draw.polygon(polygon_points, fill=255)  # 在 mask 圖像上繪製多邊形

            if label == 'cancer':
                output_folder = 'output_dir_test/masks/0'
                other_folders = ['output_dir_test/masks/1','output_dir_test/masks/2']
            elif label == 'mix':
                output_folder = 'output_dir_test/masks/1'
                other_folders =['output_dir_test/masks/0','output_dir_test/masks/2']
            elif label == 'warthin':
                output_folder = 'output_dir_test/masks/2'
                other_folders =['output_dir_test/masks/0','output_dir_test/masks/1']

            black_img = Image.new('L', (data['imageWidth'], data['imageHeight']), 0)
            # black = ImageDraw.Draw(black_img)
            b_mask_np = np.array(black_img)
            
            for i, output_f in enumerate(other_folders):
                b_output_mask_path = os.path.join(output_f, image_filename)
                print(b_output_mask_path)
                Image.fromarray(b_mask_np).save(b_output_mask_path)
                print(f"黑色 mask 已保存 {b_output_mask_path}")
            
        # 將 PIL 圖像轉換為 NumPy array
        mask_np = np.array(image)

        # 輸出的 mask 檔案路徑
        output_mask_path = os.path.join(output_folder, image_filename)

        # 將 NumPy array 保存為 PNG 檔案
        Image.fromarray(mask_np).save(output_mask_path)
        print(f"已保存 {output_mask_path}")


print("完成轉換")

output_dir_test/masks/0\b58-15991078_11.png
黑色 mask 已保存 output_dir_test/masks/0\b58-15991078_11.png
output_dir_test/masks/1\b58-15991078_11.png
黑色 mask 已保存 output_dir_test/masks/1\b58-15991078_11.png
output_dir_test/masks/0\b58-15991078_11.png
黑色 mask 已保存 output_dir_test/masks/0\b58-15991078_11.png
output_dir_test/masks/1\b58-15991078_11.png
黑色 mask 已保存 output_dir_test/masks/1\b58-15991078_11.png
已保存 output_dir_test/masks/2\b58-15991078_11.png
output_dir_test/masks/0\b59-15762303_9.png
黑色 mask 已保存 output_dir_test/masks/0\b59-15762303_9.png
output_dir_test/masks/2\b59-15762303_9.png
黑色 mask 已保存 output_dir_test/masks/2\b59-15762303_9.png
已保存 output_dir_test/masks/1\b59-15762303_9.png
output_dir_test/masks/0\b60-02986412_8.png
黑色 mask 已保存 output_dir_test/masks/0\b60-02986412_8.png
output_dir_test/masks/2\b60-02986412_8.png
黑色 mask 已保存 output_dir_test/masks/2\b60-02986412_8.png
已保存 output_dir_test/masks/1\b60-02986412_8.png
output_dir_test/masks/0\b61-08688114_6.png
黑色 mask 已保存 output_dir_